In [ ]:
import numpy as np

#haven't really tested this code too much yet, will test later
#this is code for some metrics that we can add to the isolation forest at some point
#see if they're useful

import numpy as np
import pandas as pd
from scipy.stats import linregress

def lag1_autocorrelation(time_series):
    
    """
    Calculate the lag-1 autocorrelation of a time series.
    """
    
    ts = np.array(time_series) #convert it first to numpy array so more efficient
    ts_mean = np.mean(ts)
    ts_shifted = ts[1:]  # Shifted by one, removing first element so "lagged"
    ts_original = ts[:-1]  # Original time series, excluding last element
    
    #measurement of covariance between the original values and the lagged values, relative to the mean.
    numerator = np.sum((ts_original - ts_mean) * (ts_shifted - ts_mean))
    denominator = np.sum((ts_original - ts_mean) ** 2) #variance of original time series
    return numerator / denominator if denominator != 0 else 0

def hurst_exponent(time_series):
    """
    Calculate the Hurst exponent of a time series using rescaled range analysis.
    H > 0.5 indicates persistence, H < 0.5 indicates mean-reverting behavior.
    """
    ts = np.array(time_series)
    N = len(ts)
    lags = range(2, N)  # this will consider lags up to the length of the time series of the data
    tau = [np.std(ts[lag:] - ts[:-lag]) for lag in lags] #computes standard deviation of the differences, where differences 
    #are taken between the time series shifted by "lag" steps and the original series
    
    # Linear fit to estimate the Hurst exponent
    reg = linregress(np.log(lags), np.log(tau))
    #does a linear regression, attempting to find a relationship between the log of the lag values nd the log of tau
    
    return reg.slope * 2.0  # Hurst exponent

def rise_fall_ratio_over_time(time_series):
    """
    Calculate the rise/fall ratio at every step of the time series and store it in a NumPy array.
    """
    ts = np.array(time_series)
    ratios = np.zeros(len(ts))  # Initialize an array to store the ratios
    
    # Loop through the time series from the second point onward
    for i in range(1, len(ts)):
        # Get the sub-series up to the current point
        sub_series = ts[:i+1]
        
        #this I think is a pretty clever way to calc the diff between consecutvie elements of time series
        rises = sub_series[1:] - sub_series[:-1]
        
        rise_count = np.sum(rises > 0)  # num of pos change
        fall_count = np.sum(rises < 0)  # num of neg change
        
        # Calculate the rise/fall ratio and handle division by zero
        ratios[i] = rise_count / fall_count if fall_count != 0 else np.inf
    
    # First element remains 0 (undefined ratio at step 0)
    return ratios

#try an unsupervised random forest and look at distribution of features (compare anomoalous vs normal features)
#look at correlation matrix as well
#look at feature importances (which ones are most important for isolation forest effort / most decisive)
#standard entropy (use scikit-learn might have useful implementations)